In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import os

from hazma.scalar_mediator import ScalarMediator
from hazma.pseudo_scalar_mediator import PseudoScalarMFV
from hazma.vector_mediator import VectorMediator
from hazma.parameters import neutral_pion_mass as mpi0
from hazma.cmb import f_eff_ep, f_eff_g

Masses to scan over

In [2]:
mx_min, mx_max, n_mxs = mpi0/2., 250., 100
mxs = np.linspace(mx_min, mx_max, n_mxs)

## Scalar mediator

In [3]:
s_theta = 0.5
sm_hp = ScalarMediator(mx=1., ms=1000., gsxx=1.0, gsff=s_theta, gsGG=s_theta, gsFF=-s_theta)

sm_f_effs = sm_hp.f_effs(mxs)

/Users/acoogan/.pyenv/versions/2.7.13/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)
/Users/acoogan/.pyenv/versions/2.7.13/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
/Users/acoogan/.pyenv/versions/2.7.13/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, disc

## Pseudoscalar mediator

In [9]:
psm = PseudoScalarMFV(mx=200., mp=1e3, gpxx=1., gpqq=1., gpll=0., gpGG=3., gpFF=2./3.)

psm_f_effs = psm.f_effs(mxs)

/Users/acoogan/Dropbox/Physics/Sub-GeV_DM/Hazma/hazma/rambo.py:202: RuntimeWarning: invalid value encountered in sqrt
  density=density)


In [10]:
os.system("say 'Done'")

0

## Vector mediator

In [4]:
vm = VectorMediator(mx=1., mv=1000., gvxx=1., gvuu=1., gvdd=0.2, gvss=0., gvee=0., gvmumu=0.)
vm_f_effs = vm.f_effs(mxs)  # plot for benchmark couplings

### Try different coupling ratios

In [ ]:
# Coupling ratios
ratios = np.array([-2, -1., 0., 0.25, 0.5, 0.75, 1.])

# Set up the colormap
cmap = cm.viridis_r
norm = Normalize(vmin=ratios[0], vmax=ratios[-1])
m = cm.ScalarMappable(norm=norm, cmap=cmap)
m.set_array(ratios)

vm_f_effs_gvdd = []

for ratio in ratios:
    vm.gvdd = ratio * vm.gvuu
    vm_f_effs_gvdd.append(vm.f_effs(mxs))

## Plot together

In [11]:
plt.figure(figsize=(5, 3.5))

plt.plot(mxs, vm_f_effs, label="Vector")
# for ratio, vm_f_eff in reversed(zip(ratios, vm_f_effs_gvdd)):
#     plt.plot(mxs, vm_f_eff, color=m.to_rgba(ratio))

plt.plot(mxs, sm_f_effs, label="Scalar")
plt.plot(mxs, psm_f_effs, label="Pseudoscalar")

plt.xlim(mxs[[0, -1]])
plt.ylim(0, 1.)
plt.xlabel(r"$m_\chi$ (MeV)")
plt.ylabel(r"$f_{\mathrm{eff}}^\chi$")
plt.legend()
# plt.gcf().colorbar(m, ax=plt.gca()).set_label(r"$g_{Vdd} / g_{Vuu}$")

# plt.savefig("figures/f_eff_dm.pdf")

In [8]:
os.system('say "Done computing f effective"')

0

## Electron and photon $f_{\mathrm{eff}}$s

In [27]:
plt.plot(f_eff_ep.x, f_eff_ep.y, label=r"$f_{\mathrm{eff}}^{e^+ e^-}$")
plt.plot(f_eff_g.x, f_eff_g.y, label=r"$f_{\mathrm{eff}}^\gamma$")

# plt.xlim([0.005, 250])
plt.axvline(mxs[0])
plt.axvline(mxs[-1])
plt.xscale("log")
plt.xlabel(r"$E$ (MeV)")
plt.ylabel(r"$f_{\mathrm{eff}}$")
plt.legend()

In [8]:
from hazma.cmb import vx_cmb

In [13]:
vm.mx = 150.
e_cm = 2. * vm.mx * (1. + 0.5 * vx_cmb(vm.mx, 1e-4)**2)
spec_vm = vm.spectra(f_eff_g.x, e_cm)["total"]

In [14]:
# plt.plot(f_eff_ep.x, f_eff_ep.x*f_eff_ep.y, label=r"$f_{\mathrm{eff}}^{e^+ e^-}$")
plt.plot(f_eff_g.x, f_eff_g.x*f_eff_g.y*spec_vm, label=r"$f_{\mathrm{eff}}^\gamma$")

# plt.xlim([0.005, 250])
plt.axvline(mxs[0])
plt.axvline(mxs[-1])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$E$ (MeV)")
plt.ylabel(r"$f_{\mathrm{eff}}$")
plt.legend()